In [1]:
import openreview
from deepdiff import DeepDiff
from pprint import pprint
from __future__ import print_function

guest_client = openreview.Client(baseurl="https://api.openreview.net")

def sort_by_tcdate(notes):
    if type(notes[0]) is dict:
        return list(sorted(notes, key=lambda x: x['tcdate']))
    else:
        return list(sorted(notes, key=lambda x: x.tcdate))

def print_metadata(note):
    if type(note) is dict:
        print(
            note['id'], note['forum'], 
            note['original'] if 'original' in note else None, 
            note['referent'] if 'referent' in note else None, 
            note['cdate'], note['tcdate'], note['tmdate'], note['invitation']
        )
    else:
        print(
            note.id, note.forum, note.original, note.referent,
            note.cdate, note.tcdate, note.tmdate, note.invitation
        )

In [2]:
VENUE = "ICLR.cc/2018/Conference"

In [3]:
# get all blind submissions
forum_notes = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 911


### Referent submission and comment notes

In [4]:
# get a blind submission
forum_note = forum_notes[0]
print('id forum original referent cdate tcdate tmdate invitation')
print_metadata(forum_note)
print(forum_note)

id forum original referent cdate tcdate tmdate invitation
S1sRrN-CW S1sRrN-CW BysijzbCb None 1518730155717 1509144019374 1644493125371 ICLR.cc/2018/Conference/-/Blind_Submission
{'cdate': 1518730155717,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                        'author={Jiezhong Qiu and Hao Ma and Yuxiao Dong and '
                        'Kuansan Wang and Jie Tang},\n'
                        'year={2018},\n'
                        'url={https://openreview.net/forum?id=S1sRrN-CW},\n'
                        '}',
             'abstract': 'We study the problem of knowledge base (KB) '
                         'embedding, which is usually addressed through two '
                         'frameworks---neural KB embedding and tensor '
                         'decomposition. In this work, we theoretically '
             

In [5]:
# get the referent submission (forum) note and comment notes for this blind submission
this_forum_notes = sort_by_tcdate(list(openreview.tools.iterget_notes(guest_client, forum=forum_note.id)))
print('#NotesThisBlindSubmissionForum', len(this_forum_notes))

print('id forum original referent cdate tcdate tmdate invitation')
for note in this_forum_notes:
    print_metadata(note)
    if note.id == forum_note.id:
        print(DeepDiff(note, forum_note, ignore_order=True))
    else:
        comment_note = note
assert forum_note.id in [note.id for note in this_forum_notes]

#NotesThisBlindSubmissionForum 10
id forum original referent cdate tcdate tmdate invitation
S1sRrN-CW S1sRrN-CW BysijzbCb None 1518730155717 1509144019374 1644493125371 ICLR.cc/2018/Conference/-/Blind_Submission
{}
Hkw2SfERW S1sRrN-CW None None 1509332399531 1509332399531 1509332967592 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
HJhXGX8AW S1sRrN-CW None None 1509466660293 1509466660293 1510092431411 ICLR.cc/2018/Conference/-/Paper951/Official_Comment
HkjsLOd0W S1sRrN-CW None None 1509619363323 1509619363323 1509619363323 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
BkU9F4qlf S1sRrN-CW None None 1511831950424 1511831950424 1511831950424 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
r1S26N5lG S1sRrN-CW None None 1511833004700 1511833004700 1511963280603 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
BytzlNjez S1sRrN-CW None None 1511895056605 1511895056605 1515642534971 ICLR.cc/2018/Conference/-/Paper951/Official_Review
SyeeEtTef S1sRrN-CW None None 1512047591811 151204

### Referent submission and its references

In [6]:
print("forum id\t", forum_note.id)
print("original id\t", forum_note.original)

forum id	 S1sRrN-CW
original id	 BysijzbCb


In [8]:
# original submission and its revisions
submission_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=True))
print('#References:', len(submission_references_oT))

has_original = False
print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oT:
    assert ref.original == forum_note.original
    print_metadata(ref)
    if ref.id == forum_note.original:
        print("above is the original submission note")
        has_original = True
    # TODO check that the original has the same contents as the forum
    # TODO only need one of this and the forum note
if not has_original:
    print("MISSING THE ORIGINAL NOTE")

#References: 2
id forum original referent cdate tcdate tmdate invitation
BysijzbCb None BysijzbCb S1sRrN-CW None 1509137319414 1516894791255 ICLR.cc/2018/Conference/-/Blind_Submission
SpGpHFdfkq None BysijzbCb S1sRrN-CW None 1644493125371 1644493125371 ICLR.cc/2018/Conference/-/Blind_Submission


In [27]:
# this seems to return one of the above revisions, which is Code
guest_client.get_all_references(referent=forum_note.original, original=True)

[Note(id = 'SpGpHFdfkq',original = None,number = 494,cdate = 1644493125371,mdate = 1644493125371,tcdate = 1644493125371,tmdate = 1644493125371,ddate = None,content = {'data': '[FB15k](https://paperswithcode.com/dataset/fb15k), [WN18](https://paperswithcode.com/dataset/wn18)'},forum = 'BysijzbCb',referent = 'BysijzbCb',invitation = 'PapersWithCode.com/-/ICLR_2018_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = {})]

In [28]:
guest_client.get_all_references(referent=forum_note.original, original=False)

[Note(id = 'SpGpHFdfkq',original = None,number = 494,cdate = 1644493125371,mdate = 1644493125371,tcdate = 1644493125371,tmdate = 1644493125371,ddate = None,content = {'data': '[FB15k](https://paperswithcode.com/dataset/fb15k), [WN18](https://paperswithcode.com/dataset/wn18)'},forum = 'BysijzbCb',referent = 'BysijzbCb',invitation = 'PapersWithCode.com/-/ICLR_2018_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = None)]

In [9]:
print(forum_note)

{'cdate': 1518730155717,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                        'author={Jiezhong Qiu and Hao Ma and Yuxiao Dong and '
                        'Kuansan Wang and Jie Tang},\n'
                        'year={2018},\n'
                        'url={https://openreview.net/forum?id=S1sRrN-CW},\n'
                        '}',
             'abstract': 'We study the problem of knowledge base (KB) '
                         'embedding, which is usually addressed through two '
                         'frameworks---neural KB embedding and tensor '
                         'decomposition. In this work, we theoretically '
                         'analyze the neural embedding framework and '
                         'subsequently connect it with tensor based embedding. '
                         'Specifically,

In [11]:
print(submission_references_oT[0]) # original

{'cdate': None,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                        'author={Jiezhong Qiu and Hao Ma and Yuxiao Dong and '
                        'Kuansan Wang and Jie Tang},\n'
                        'year={2018},\n'
                        'url={https://openreview.net/forum?id=S1sRrN-CW},\n'
                        '}',
             'abstract': 'We study the problem of knowledge base (KB) '
                         'embedding, which is usually addressed through two '
                         'frameworks---neural KB embedding and tensor '
                         'decomposition. In this work, we theoretically '
                         'analyze the neural embedding framework and '
                         'subsequently connect it with tensor based embedding. '
                         'Specifically, we show 

In [12]:
print(submission_references_oT[1]) # revision

{'cdate': None,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                        'author={Jiezhong Qiu and Hao Ma and Yuxiao Dong and '
                        'Kuansan Wang and Jie Tang},\n'
                        'year={2018},\n'
                        'url={https://openreview.net/forum?id=S1sRrN-CW},\n'
                        '}',
             'data': '[FB15k](https://paperswithcode.com/dataset/fb15k), '
                     '[WN18](https://paperswithcode.com/dataset/wn18)'},
 'ddate': None,
 'details': {'original': {'cdate': 1644493125371,
                          'content': {'data': '[FB15k](https://paperswithcode.com/dataset/fb15k), '
                                              '[WN18](https://paperswithcode.com/dataset/wn18)'},
                          'ddate': None,
                          'forum': 'Bysijzb

### A referent comment note and its references

In [13]:
comment_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=True))
print('#References:', len(comment_references_oT))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oT:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
r1uxS1arz None None r1uxS1arz None 1517249775749 1517260087457 ICLR.cc/2018/Conference/-/Acceptance_Decision


In [14]:
comment_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=False))
print('#References:', len(comment_references_oF))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oF:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
r1uxS1arz None None r1uxS1arz None 1517249775749 1517260087457 ICLR.cc/2018/Conference/-/Acceptance_Decision


In [15]:
print(comment_note)

{'cdate': 1517249775732,
 'content': {'comment': 'The reviewers are not convinced by a number of '
                        'aspects: including originality and clarity. Whereas '
                        'the assessment of clarity and originality may be '
                        'somewhat subjective (though the connections between '
                        'margin-based loss and negative sampling is indeed '
                        'well known), it is pretty clear that evaluation is '
                        'very questionable. This is not so much about '
                        'existence of more powerful factorizations  (e.g., '
                        'ConvE / HolE) but the fact that the shown baselines '
                        '(e.g., DistMult) can be tuned to yield much better '
                        'performance on these benchmarks.  Also, indeed the '
                        'authors should report results on cleaned versions of '
                        'the datasets (e.g., FB1

In [16]:
print(comment_references_oT[0])

{'cdate': None,
 'content': {'comment': 'The reviewers are not convinced by a number of '
                        'aspects: including originality and clarity. Whereas '
                        'the assessment of clarity and originality may be '
                        'somewhat subjective (though the connections between '
                        'margin-based loss and negative sampling is indeed '
                        'well known), it is pretty clear that evaluation is '
                        'very questionable. This is not so much about '
                        'existence of more powerful factorizations  (e.g., '
                        'ConvE / HolE) but the fact that the shown baselines '
                        '(e.g., DistMult) can be tuned to yield much better '
                        'performance on these benchmarks.  Also, indeed the '
                        'authors should report results on cleaned versions of '
                        'the datasets (e.g., FB15k-237). 

In [17]:
print(comment_references_oF[0])

{'cdate': None,
 'content': {'comment': 'The reviewers are not convinced by a number of '
                        'aspects: including originality and clarity. Whereas '
                        'the assessment of clarity and originality may be '
                        'somewhat subjective (though the connections between '
                        'margin-based loss and negative sampling is indeed '
                        'well known), it is pretty clear that evaluation is '
                        'very questionable. This is not so much about '
                        'existence of more powerful factorizations  (e.g., '
                        'ConvE / HolE) but the fact that the shown baselines '
                        '(e.g., DistMult) can be tuned to yield much better '
                        'performance on these benchmarks.  Also, indeed the '
                        'authors should report results on cleaned versions of '
                        'the datasets (e.g., FB15k-237). 

## Other exploration

### Get notes with direct or all replies

In [18]:
# get all blind submissions, each with its direct replies
forum_notes_dr = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='directReplies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 911


In [19]:
# get all blind submissions, each with its replies
forum_notes_r = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='replies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 911


In [20]:
# the first blind submission with direct replies
forum_note = forum_notes_dr[0]
print('#directReplies:', len(forum_note.details['directReplies']))
for reply in sort_by_tcdate(forum_note.details['directReplies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#directReplies: 5
Hkw2SfERW S1sRrN-CW None None 1509332399531 1509332399531 1509332967592 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
BytzlNjez S1sRrN-CW None None 1511895056605 1511895056605 1515642534971 ICLR.cc/2018/Conference/-/Paper951/Official_Review
SyeeEtTef S1sRrN-CW None None 1512047591811 1512047591811 1515642534932 ICLR.cc/2018/Conference/-/Paper951/Official_Review
BkwNvgRgf S1sRrN-CW None None 1512077102815 1512077102815 1515642534895 ICLR.cc/2018/Conference/-/Paper951/Official_Review
r1uxS1arz S1sRrN-CW None None 1517249775732 1517249775749 1517260087457 ICLR.cc/2018/Conference/-/Acceptance_Decision

S1sRrN-CW S1sRrN-CW BysijzbCb None 1518730155717 1509144019374 1644493125371 ICLR.cc/2018/Conference/-/Blind_Submission
{'cdate': 1518730155717,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                     

In [21]:
# the first blind submission with all replies
forum_note = forum_notes_r[0]
print('#replies:', len(forum_note.details['replies']))
for reply in sort_by_tcdate(forum_note.details['replies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#replies: 9
Hkw2SfERW S1sRrN-CW None None 1509332399531 1509332399531 1509332967592 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
HJhXGX8AW S1sRrN-CW None None 1509466660293 1509466660293 1510092431411 ICLR.cc/2018/Conference/-/Paper951/Official_Comment
HkjsLOd0W S1sRrN-CW None None 1509619363323 1509619363323 1509619363323 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
BkU9F4qlf S1sRrN-CW None None 1511831950424 1511831950424 1511831950424 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
r1S26N5lG S1sRrN-CW None None 1511833004700 1511833004700 1511963280603 ICLR.cc/2018/Conference/-/Paper951/Public_Comment
BytzlNjez S1sRrN-CW None None 1511895056605 1511895056605 1515642534971 ICLR.cc/2018/Conference/-/Paper951/Official_Review
SyeeEtTef S1sRrN-CW None None 1512047591811 1512047591811 1515642534932 ICLR.cc/2018/Conference/-/Paper951/Official_Review
BkwNvgRgf S1sRrN-CW None None 1512077102815 1512077102815 1515642534895 ICLR.cc/2018/Conference/-/Paper951/Official_Review
r1uxS1a

### Get references with other settings

In [24]:
# referent=forum_note.id, original=False -> this returns the submission/forum note with little content
submission_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=False))
print('#References:', len(submission_references_oF))
assert len(submission_references_oF) == 1

print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oF:
    assert ref.id == forum_note.id
    assert ref.original == forum_note.original
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
S1sRrN-CW None BysijzbCb S1sRrN-CW None 1509144019374 1518730155745 ICLR.cc/2018/Conference/-/Blind_Submission


In [10]:
print(submission_references_oF[0])

{'cdate': None,
 'content': {'_bibtex': '@misc{\n'
                        'qiu2018revisiting,\n'
                        'title={Revisiting Knowledge Base Embedding as Tensor '
                        'Decomposition},\n'
                        'author={Jiezhong Qiu and Hao Ma and Yuxiao Dong and '
                        'Kuansan Wang and Jie Tang},\n'
                        'year={2018},\n'
                        'url={https://openreview.net/forum?id=S1sRrN-CW},\n'
                        '}'},
 'ddate': None,
 'details': None,
 'forum': None,
 'id': 'S1sRrN-CW',
 'invitation': 'ICLR.cc/2018/Conference/-/Blind_Submission',
 'mdate': None,
 'nonreaders': [],
 'number': None,
 'original': 'BysijzbCb',
 'readers': ['everyone'],
 'referent': 'S1sRrN-CW',
 'replyto': None,
 'signatures': ['ICLR.cc/2018/Conference'],
 'tcdate': 1509144019374,
 'tmdate': 1518730155745,
 'writers': ['ICLR.cc/2018/Conference']}


### Seemingly useful but actually confusing

In [22]:
# get venue-specific event types
# the problem is that, unexpectedly, this doesn't return Official_Review
invitations = openreview.tools.iterget_invitations(
        guest_client, regex=f'{VENUE}.*'
)
event_types = set()
for invitation in invitations:
    event_types.add(invitation.id.split('/')[-1])
print(event_types)

{'Official_Comment', 'Blind_Submission', 'Add_Revision', 'Public_Comment', 'Withdrawn_Submission', 'Revise_Review', 'Withdraw_Paper'}
